In [1]:
import sys
import os
import requests
import json

In [2]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_rknop_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [3]:
rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [4]:
# Depends on global variable rqs
def run_query( query, subdict={} ):
    result = rqs.post( f'{url}/db/runsqlquery/', json={ 'query': query, 'subdict': subdict } )
    if ( result.status_code != 200 ):
        raise RuntimeError( f"Got http status {result.status_code}" )
    data = json.loads( result.text )
    if ( 'status' in data ) and ( data['status'] == 'error' ):
        sys.stderr.write( f"ERROR: {data['error']}" )
        return []
    elif ( 'status' not in data ) or ( data['status'] != 'ok' ):
        raise RuntimeError( f"Got unexpected response: {data}" )
    return data['rows']


In [5]:
query = 'SELECT "classId",snana_gentype,description FROM elasticc_classificationmap'
rows = run_query( query )
print( f'classId   gentype    description' )
print( '---------------------------------' )
for row in rows:
    print( f'{row["classId"]:7d}   '
           f'{"None" if row["snana_gentype"] is None else str(row["snana_gentype"]):>7s}    '
           f'{row["description"]}' )

classId   gentype    description
---------------------------------
      0      None    Static/Other
      1      None    Non-Recurring
     10      None    Non-Recurring/Other
     11      None    SN-like
    110      None    SN-like/Other
    111        10    Ia
    112        20    Ib/c
    112        21    Ib/c
    112        25    Ib/c
    112        26    Ib/c
    112        27    Ib/c
    113        30    II
    113        31    II
    113        32    II
    113        35    II
    113        36    II
    113        37    II
    114        12    Iax
    115        11    91bg
     12      None    Fast
    120      None    Fast/Other
    121        50    KN
    121        51    KN
    122        82    M-dwarf Flare
    123        84    Dwarf Nova
    124        87    uLens
    124        88    uLens
    124        89    uLens
     13      None    Long
    130      None    Long/Other
    131        40    SLSN
    132        42    TDE
    133        45    ILOT
    134        46    

In [11]:
query = 'SELECT * FROM elasticc_diasource WHERE "diaObject_id"=%(id)s ORDER BY "midPointTai"'
subdict = { "id": 6018461 } # { "id": 10208128  }
rows = run_query( query, subdict )

In [12]:
sys.stderr.write( f"Got {len(rows)} rows" )
for row in rows:
    print( f'Object: {row["diaObject_id"]:<20d}, Source: {row["diaSourceId"]:<20d}, '
           f'MJD: {row["midPointTai"]:9.2f}, Flux: {row["psFlux"]:10.2f}' )

Object: 6018461             , Source: 12036922062         , MJD:  60636.04, Flux:     677.09
Object: 6018461             , Source: 12036922064         , MJD:  60638.18, Flux:     674.40
Object: 6018461             , Source: 12036922106         , MJD:  60990.08, Flux:    1283.54
Object: 6018461             , Source: 12036922107         , MJD:  60990.10, Flux:    1008.38
Object: 6018461             , Source: 12036922110         , MJD:  60993.23, Flux:    1116.54
Object: 6018461             , Source: 12036922118         , MJD:  61028.16, Flux:    1119.00
Object: 6018461             , Source: 12036922147         , MJD:  61322.32, Flux:     882.81
Object: 6018461             , Source: 12036922176         , MJD:  61342.08, Flux:     762.49
Object: 6018461             , Source: 12036922182         , MJD:  61354.04, Flux:    1036.80
Object: 6018461             , Source: 12036922183         , MJD:  61354.06, Flux:     883.87
Object: 6018461             , Source: 12036922184         , MJD:  6135

Got 21 rows

In [29]:
data

{'status': 'ok',
 'rows': [{'diaSourceId': 12036922062,
   'ccdVisitId': -1,
   'parentDiaSourceId': None,
   'midPointTai': 60636.0428,
   'filterName': 'g',
   'ra': 28.96840362195947,
   'decl': -21.886205596973067,
   'psFlux': 677.0934448242188,
   'psFluxErr': 78.85809326171875,
   'snr': 8.586226463317871,
   'nobs': 224.0,
   'diaObject_id': 6018461},
  {'diaSourceId': 12036922064,
   'ccdVisitId': -1,
   'parentDiaSourceId': None,
   'midPointTai': 60638.1793,
   'filterName': 'g',
   'ra': 28.96840362195947,
   'decl': -21.886205596973067,
   'psFlux': 674.39990234375,
   'psFluxErr': 100.12347412109375,
   'snr': 6.735682487487793,
   'nobs': 224.0,
   'diaObject_id': 6018461},
  {'diaSourceId': 12036922106,
   'ccdVisitId': -1,
   'parentDiaSourceId': None,
   'midPointTai': 60990.0784,
   'filterName': 'g',
   'ra': 28.96840362195947,
   'decl': -21.886205596973067,
   'psFlux': 1283.5400390625,
   'psFluxErr': 101.55677032470703,
   'snr': 12.63864517211914,
   'nobs': 22